# Week 02 Assignment glucose level data


Missing data occurs commonly in many data applications. Especially wearable sensor devices encounter measurement errors resulting in missing data. Although pandas can handle missing data, for instance, to exclude missing data in the descriptive analysis, we might decide to reshape our data to improve quality. We might consider excluding certain rows, columns, or decide to impute our data if we take an argumentative approach. This week we will work with sensor data of a semester 01 project. The data is timeseries related but a lot of sensor errors occurred. First, we inspect the quality data. Then we will work with imputation, interpolation, and smoothing techniques to overcome this issue of the error readings. Finaly we plot the data. Visualization is a method to translate data into information. As a data scientist you should be able to choose an appropiate visualizations. 


Keywords: data loading, data inspection, data exploration, data cleaning, impute data, missing data, timeseries, pandas, visualization, interactive plots, overflow/underflow error

More to read
- https://fennaf.gitbook.io/bfvm22prog1/
- https://towardsdatascience.com/how-to-analyze-blood-glucose-data-with-python-data-science-packages-4f160f9564be


More about pandas and bokeh programming
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
- https://rubberduckdebugging.com/
- https://docs.bokeh.org/en/latest/docs/gallery.html


Learning objectives

- Further practice numpy / pandas
- Read, inspect, clean, and impute JSON data. Handle missing data
- Apply different interpolation/smoothing techniques
- Recognize overflow/underflow error
- Perform visual and statistical analysis for time series data
- Maintain development environment 
- Apply coding standards and FAIR principles


Note: you can bring your own sensor data, but for the practical you need some missing data points. 

By the end of this week the student can:

- load a json dataset 
- typecast the Pandas DataFrame to appropiate data types
- inspect the dataset for quality and metadata information
- add a column with interpolated data in Pandas DataFrame
- perform visual analysis

please add your own topics you want to learn here: https://padlet.com/ffeenstra1/69vbvy0l8t32rq55


The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: prepare for inspection](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: inspect the data](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: interpolate the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: visualize the data](#4)
     - [Exercise 5.1](#ex-51)
- [part 6: Challenge](#5)
     - [Exercise 6.1](#ex-61)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
Mind you that you cannot copy code without referencing the code. If you copy code you need to be able to explain your code verbally and you will not get the full score. 


<a name='0'></a>
## Part 1: Load the data

Instructions: Load the json datafile `glucose.json` into a pandas dataframe. Check your dataframe with a `.head()` to compare with the expected outcome. 

Preferably we read the data not with a hard coded data path but using a config file. See https://fennaf.gitbook.io/bfvm22prog1/data-processing/configuration-files/yaml

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>json.load() method reads a file, pd.read_json converts it to a Pandas DataFrame</li>
    <li>when loading into a Pandas DataFrame use records orientation </li>
</ul>
</details>

<a name='ex-11'></a>
### Code your solution

In [1]:
import yaml

def get_config():
    """ function that fetches the configuration 
    parameters from config.yaml"""
    with open("./config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()
glucose = config['glucose_data']

In [2]:
# CODE YOUR SOLUTION HERE
import pandas as pd
import json

with open(glucose) as inputfile:
    df = pd.read_json(json.load(inputfile), orient='records')

print(df.head())

       ID              time  recordtype glucose
0  2845.0  2019-04-25 00:08           1     109
1  2850.0  2019-04-25 00:50           1        
2  2877.0  2019-04-25 07:02           1     123
3  2881.0  2019-04-25 07:34           1     158
4  2886.0  2019-04-25 08:19           1        


#### Expected outcome: 

<a name='1'></a>
## Part 2: Prepare the data

Check the datatypes of your dataframe. The `glucose` field should be an integer, the `time` field should have a datetime format. If the datatypes are different you should typecast them to the right format.
Make sure that your dataset is sorted by the time column


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use astype() method or pandas.DataFrame.to_datetime() for instance</li>
    <li>make sure that the empty spaces are filled with NaN. Use errors='coerce'</li>
    <li>set_index(), sort_index() and reset_index() are helpful to sort on index</li>
</ul>
</details>

<a name='ex-21'></a>
### Code your solution

In [3]:
# CODE YOUR SOLUTION HERE
df = df.astype({'ID': 'int64', 'time': 'datetime64[ns]'})
df.glucose = pd.to_numeric(df.glucose, errors='coerce')  
df = df.sort_values(by=['time'])
print(df.dtypes)

ID                     int64
time          datetime64[ns]
recordtype             int64
glucose              float64
dtype: object


#### Expected outcome: 

<a name='2'></a>
## Part 3: Inspect the data

Now that we prepared the data we are going to inspect the data to get more familiar with the data. You can answer the questions below but feel free to explore the data further by plotting and or statistics, group by overviews and so on. 

To be answered at least:
- what is the quantity of the data (observations and features)
- what is the percentage missing data for glucose?
- is there a specific pattern for glucose value and recordtype?
- what is the time range?

Code the solutions to your answers. Create meaningful overviews or statistics

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>In the week 01 assignment some functions were explained to inspect missing values</li>
    <li>In the week 01 assignment some functions were explained to groupby value</li>
</ul>
</details>

<a name='ex-31'></a>
### Code your solution

In [4]:
# CODE YOUR SOLUTION HERE
# what is the quantity of the data:
print(f'number of observations: {len(df.axes[0])}')
print(f'number of features: {len(df.axes[1])}', '\n')

# what is the percentage missing data for glucose
missng_perc =  df.glucose.isnull().sum() / len(df.glucose) * 100
print(f'missing glucose data: {missng_perc} %', '\n')  

# is there a specific pattern for glucose value and recordtype
missing_values = df.glucose.isnull().groupby(df.recordtype).sum()
total_values = df.glucose.groupby(df.recordtype).size()
print(f'missing glucose data with recordtype 0: {missing_values[0] / total_values[0] * 100} %')
print(f'missing glucose data with recordtype 1: {missing_values[1] / total_values[1] * 100} %')
print('--> all of the glucose data with recordtype 0 are missing, this is not the case with recordtype 1 where it is partially missing.', '\n')

print(f'time range: {df.time.min()} - {df.time.max()}')

number of observations: 136
number of features: 4 

missing glucose data: 61.76470588235294 % 

missing glucose data with recordtype 0: 100.0 %
missing glucose data with recordtype 1: 3.7037037037037033 %
--> all of the glucose data with recordtype 0 are missing, this is not the case with recordtype 1 where it is partially missing. 

time range: 2019-04-25 00:08:00 - 2019-04-25 23:48:00


#### Expected outcome percentage missing data
0.6176470588235294

<a name='3'></a>
## Part 4: Interpolate the data

A lot of data is missing. Use interpolation to fill the missing values. Create a new column with the interpolated data. Take an argumentative approach. Select an interpolation method that suits the nature of the data and explain your choice. Mind you that the expected outcome of the interpolation values can differ from the example below

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use Pandas.DataFrame.interpolate() method</li>
</ul>
</details>

<a name='ex-41'></a>
### Code your solution

In order to fill in the missing values, interpolation is uses. The method 'linear' is chosen after the data is sorted by time and date. The reason for this is that it is likely that the data of glucose measurements to linearly increase or decrease between to known values. However, it can be that there are some fluctations that are not shown. Especially when there is a lot of data missing between two datapoints. This should be kept in mind when using this data for further analsysis 

In [5]:
#CODE YOUR SOLUTION HERE
df = df.set_index(['time']).sort_index().reset_index()
df['interpolated'] = df.glucose.interpolate(method='linear')
print(df.head())

                 time                   ID  recordtype  glucose  interpolated
0 2019-04-25 00:08:00                 2845           1    109.0    109.000000
1 2019-04-25 00:14:00 -9223372036854775808           0      NaN    109.466667
2 2019-04-25 00:29:00 -9223372036854775808           0      NaN    109.933333
3 2019-04-25 00:44:00 -9223372036854775808           0      NaN    110.400000
4 2019-04-25 00:50:00                 2850           1      NaN    110.866667


#### Example outcome

<a name='4'></a>
## Part 5: Plot the data

Create a plot with the original data and the interpolated data. Consider what the best representation is for visualisation of actual values and modelled/imputed values. Provide argumentation for the plot of choice. 

Mind you, there are several types of plots. The most used are: 
- Compare values->	Bar, boxplot, violin plot, scatter, line. 
- Show composition-> Stacked bar, pie, mosaic, area, burtin.
- Show distribution-> 	Histogram, density, qqplot, bubble. 
- Analyzing trend-> Line, bar, dual axis 
- Relations-> Scatter, heatmap


### Code your solution

In [6]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, curdoc
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [7]:
#CODE YOUR SOLUTION HERE
# use dark theme
curdoc().theme = 'dark_minimal'

# create figure
p = figure(width=800, height=500,
           title='Scatterplot of (interpolated) glucose sensor data \n',
           x_axis_label='time',
           x_axis_type='datetime',
           y_axis_label='glucose (mg/dL)')
p.title.align = 'center'

# create figure objects
p.circle(df.time, df.interpolated, 
         size=10, color='deepskyblue', alpha=0.3,
         legend_label='interpolated (linear) glucose data')
p.circle(df.time, df.glucose, 
          size=4, color='lime', alpha=1, 
          legend_label='glucose data')

p.legend.location = 'top_left'
p.title.text_font_size = "18px"

# show plot
show(p)

As shown in the figure above the data is all over the place. The interpolated data show, especially in the beginning, the possible trends. However, since the data is probably inaccurate the interpolated data is also less reliable. 

<a name='6'></a>
## Part 6: Challenge

It might even be interesting to introduce a widget in which you can select different methods to interpolate.
1. Can you improve the interpolation by choosing an other method?
2. Can you add an rolling mean line? 
2. Can you improve the plot by making it interactive?

<a name='ex-61'></a>
### Code your solution

In [8]:
#CODE YOUR SOLUTION HERE
from bokeh.io import show, curdoc
from bokeh.models import CheckboxGroup, ColumnDataSource, Select
from bokeh.layouts import column, row
from bokeh.plotting import figure
import panel as pn

output_notebook()
pn.extension()

Loading BokehJS ...

In [9]:
# all of the possible interpolation methods:
interpolate_methods = ['linear', 'time', 'index', 'pad', 'nearest', 'zero', 'slinear', 'quadratic',
                       'cubic', 'spline', 'barycentric', 'polynomial','krogh', 'piecewise_polynomial', 
                       'spline','pchip', 'akima', 'cubicspline', 'from_derivatives']

# create a df column for every interpolation method (if possible)
for method in interpolate_methods:
    try:
        df[method] = df.glucose.interpolate(method=method)
    except ValueError:  
        continue  # cotinue of the method is not possible

In [10]:
def render_plot(method='linear', rolling_mean_window=1):
    """Function to render a bokeh plot"""
    # create figure
    p = figure(width=800, height=500,
               title='Scatterplot of (interpolated) glucose sensor data \n',
               x_axis_label='time',
               x_axis_type='datetime',
               y_axis_label='glucose (mg/dL)')
    p.title.align = 'center'

    # create figure objects
    p.line(df.time, df[method].rolling(rolling_mean_window).mean(),
              color='white', legend_label='rolling mean line')
    p.circle(df.time, df[method], 
             size=10, color='deepskyblue', alpha=0.4,
             legend_label=f'interpolated ({method}) glucose data')
    p.circle(df.time, df.glucose, 
              size=4, color='lime', alpha=1, 
              legend_label='original glucose data')
    
    # add legend and title
    p.legend.location = 'top_left'
    p.title.text_font_size = "18px"
    return p

In [11]:
# create widgets
inter_methods = pn.interact(render_plot, method=df.columns[5:], rolling_mean_window=[i for i in range(21)])

# create dashboard and add widgets
dashboard = pn.template.BootstrapTemplate(title='Glucose sensor data', sidebar_width = 350, 
                                          theme = pn.template.theme.DarkTheme)
dashboard.sidebar.append(inter_methods[0])
dashboard.main.append(inter_methods[1])

# show dashboard
dashboard.show()

Launching server at http://localhost:36473
